In [1]:
import pandas as pd

In [77]:
users = pd.read_csv("users.csv")
repos = pd.read_csv("./repositories.csv")

In [4]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360 entries, 0 to 359
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   login         360 non-null    object
 1   name          341 non-null    object
 2   company       233 non-null    object
 3   location      360 non-null    object
 4   email         252 non-null    object
 5   hireable      101 non-null    object
 6   bio           272 non-null    object
 7   public_repos  360 non-null    int64 
 8   followers     360 non-null    int64 
 9   following     360 non-null    int64 
 10  created_at    360 non-null    object
dtypes: int64(3), object(8)
memory usage: 31.1+ KB


#### Q1: Who are the top 5 users in Beijing with the highest number of followers? List their login in order, comma-separated.

In [12]:
top_5_followers_users = users.iloc[users["followers"].nlargest(
    5).index]["login"]

top_5_followers_users

0        michaelliao
1           daimajia
2            xiaolai
3          draveness
4    hongyangAndroid
Name: login, dtype: object

In [13]:
",".join(top_5_followers_users)

'michaelliao,daimajia,xiaolai,draveness,hongyangAndroid'

#### Q2: Who are the 5 earliest registered GitHub users in Beijing? List their login in ascending order of created_at, comma-separated.

In [21]:
earliest_5_users = users.iloc[pd.to_datetime(
    users["created_at"]).nsmallest(5).index]["login"]
earliest_5_users

295           robin
111           nwind
125           reeze
72            kejun
98     ZhangHanDong
Name: login, dtype: object

In [22]:
",".join(earliest_5_users)

'robin,nwind,reeze,kejun,ZhangHanDong'

#### Q3: What are the 3 most popular license among these users? Ignore missing licenses. List the license_name in order, comma-separated.

In [24]:
repos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29561 entries, 0 to 29560
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   login             29561 non-null  object
 1   full_name         29561 non-null  object
 2   created_at        29561 non-null  object
 3   stargazers_count  29561 non-null  object
 4   watchers_count    29561 non-null  object
 5   language          20759 non-null  object
 6   has_projects      29561 non-null  bool  
 7   has_wiki          29561 non-null  bool  
 8   license_name      16070 non-null  object
dtypes: bool(2), object(7)
memory usage: 1.6+ MB


In [52]:
top_3_license = repos["license_name"].value_counts(
    dropna=False).nlargest(3).index
top_3_license

Index([nan, 'mit', 'apache-2.0'], dtype='object', name='license_name')

In [55]:
",".join([str(license) for license in top_3_license])

'nan,mit,apache-2.0'

#### Q4: Which company do the majority of these developers work at?

In [56]:
users["company"].value_counts(dropna=False)

company
NaN                    127
BYTEDANCE               12
ALIBABA                  9
PEKING UNIVERSITY        7
TSINGHUA UNIVERSITY      7
                      ... 
FMSOFT                   1
TENCENTARC               1
RESIX-OS                 1
BUPT                     1
MT                       1
Name: count, Length: 180, dtype: int64

#### Q5: Which programming language is most popular among these users?

In [59]:
repos["language"].value_counts(dropna=False)

language
NaN                    8802
JavaScript             4446
Python                 3282
Java                   2197
Go                     1361
                       ... 
M4                        1
NSIS                      1
Pug                       1
Earthly                   1
Game Maker Language       1
Name: count, Length: 144, dtype: int64

#### Q6: Which programming language is the second most popular among users who joined after 2020?

In [61]:
users_after_2020 = users[pd.to_datetime(users["created_at"]).dt.year > 2020]

In [63]:
repos[repos["login"].isin(users_after_2020["login"])
      ]["language"].value_counts(dropna=False)

language
Python              10
NaN                  9
HTML                 6
TypeScript           5
C++                  3
JavaScript           3
Jupyter Notebook     3
Astro                2
Vue                  2
C#                   2
Name: count, dtype: int64

#### Q7: Which language has the highest average number of stars per repository?

In [78]:
repos["stargazers_count"].value_counts()

stargazers_count
0       16458
1        3877
2        1619
3         785
4         584
        ...  
1019        1
4979        1
618         1
979         1
6242        1
Name: count, Length: 1098, dtype: int64

In [81]:
repos.groupby("language")["stargazers_count"].mean().nlargest(10)

language
Jinja         3420.000000
Solidity      1286.666667
VBScript       698.000000
Lex            643.000000
TeX            294.222222
Stylus         232.400000
TypeScript     205.325088
Java           190.801092
MATLAB         189.840000
VBA            182.000000
Name: stargazers_count, dtype: float64

#### Q8: Let's define `leader_strength` as `followers / (1 + following)`. Who are the top 5 in terms of leader_strength? List their login in order, comma-separated.

In [90]:
users["leader_strength"] = users[["followers", "following"]].apply(
    lambda x: x["followers"] / (1 + x["following"]), axis=1)

In [95]:
top_5_leaders = users.iloc[users["leader_strength"].nlargest(5).index]["login"]
top_5_leaders

0     michaelliao
9       ityouknow
14    liuhuanyong
32         thunlp
35        shenghy
Name: login, dtype: object

In [96]:
",".join(top_5_leaders)

'michaelliao,ityouknow,liuhuanyong,thunlp,shenghy'